In [ ]:
import sys
sys.path.append('/Users/jiechen/Apoex/Email/fastAPI')
from app.services.utils import fetchFromDB, dataframe_to_model, model_to_dataframe
from app.services.base_service import BaseService
from app.dataset.email_dataset import EmailDataset
import pandas as pd

bs = BaseService()
# id = 194949
# email = fetchFromDB(bs.email_spec_query.format(EMAILID=id))
# email = pd.read_json(f'data/test_data/email8.json')
# email.to_json(f'data/test_data/cate{id}.json', orient="records", indent=4, force_ascii=False)
# email = pd.read_json(f'data/test_data/email8.json')
# emails = dataframe_to_model(email, EmailIn)
# ed = EmailDataset(df=email)
# ca = ed.do_connect()
# ca

,id,from,sender,source,to,receiver,category,errandId,totalAmount,settlementAmount,reference,insuranceCaseRef,insuranceNumber,damageNumber,animalName,ownerName,note,connectedCol,showPage,isStaffAnimal
0,189142,djurskador@folksam.se,Folksam,Insurance_Company,klinik-djurhalsan@direktregleringsportalen.se,Djurhälsan i Dalarna,Settlement_Approved,[72686],4805.0,2254.0,1000723682,1000723682,FF093480101S,FF093480101S,Murre,PIA LENA ELFGREN,Reliable,reference,Ersätt,False
1,189145,Dir.HundKatt@dina.se,Dina Försäkringar,Insurance_Company,klinik-halsingevetsoderhamn@direktregleringspo...,Hälsinge Smådjursklinik Söderhamn,Settlement_Approved,[72545],1721.0,0.0,1000722270,1000722270,263454693,<NA>,Ruff,Anna Hanold,Reliable,reference,Koppla,False
2,189147,djurskador@folksam.se,Folksam,Insurance_Company,klinik-cityvet@direktregleringsportalen.se,Cityveterinären Malmö,Settlement_Approved,[72623],375.0,281.0,1000723054,1000723054,FF093171882S,FF093171882S,Teddi,LISELOTT SANDSTRÖM,Reliable,reference,Ersätt,False
3,189149,djurskador@if.se,If,Insurance_Company,klinik-halsingevethudik@direktregleringsportal...,Hälsinge Smådjursklinik Hudiksvall,Manual_Handling_Required,[],<NA>,<NA>,66821,<NA>,<NA>,251 744 699,Lilla Stjärnan,Larsson Jens Daniel,Unreliable,None,Koppla,False
4,189150,direktreglering@lassie.co,Lassie,Insurance_Company,klinik-djurklinikkiruna@direktregleringsportal...,Lapplands Djurklinik Kiruna,Settlement_Approved,[72691],3085.0,<NA>,1000723732,1000723732,VOFF-VALTER-220607-19,072425-44627-60*,<NA>,Ida Rova,Reliable,reference,Ersätt,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,191488,djurskador@svedea.se,Svedea,Insurance_Company,"klinik-dingle@direktregleringsportalen.se, t.m...",Dingleveterinärerna,Settlement_Approved,[73531],7943.0,5957.0,1000732139,1000732139,5843439,HU25-5843439-11954,Hedda,RETURADRESS,Reliable,reference,Ersätt,False
854,191489,erik.hadfy@apoex.se,DRP,DRP,mail@drp.se,DRP,Manual_Handling_Required,[],<NA>,<NA>,3877473579,<NA>,<NA>,<NA>,<NA>,<NA>,Unreliable,None,Koppla,False
855,191500,djurskador@svedea.se,Svedea,Insurance_Company,"ann_sofieekman@hotmail.com, klinik-djurklinikl...",Lapplands Djurklinik Luleå,Settlement_Approved,[73586],5160.0,3870.0,1000732683,1000732683,8360759,HU25-8360759-13836,Musse (Klubban),RETURADRESS,Reliable,reference,Ersätt,False
856,191502,djurskador@folksam.se,Folksam,Insurance_Company,klinik-halsingevethudik@direktregleringsportal...,Hälsinge Smådjursklinik Hudiksvall,Settlement_Approved,[73588],2342.0,1618.0,1000732709,1000732709,FF093344760S,FF093344760S,Disel,STINA HAGLUNDH,Reliable,reference,Ersätt,False


In [4]:
from app.services.classifier import Classifier
a = Classifier()
a.statistic(ca)

all_records: 858, auto_categorized: 810, 94.41%, manual: 48, 5.59%, 
                  connected_to_errands: 710, 82.75%, single_errandId: 710, 82.75%,                   reliable_connection: 622, 72.49%, staff_animal: 0, 0.00%
 - Auto_Reply: 71, connect:47, un-connect: 24
 - Finance_Report: 6, connect:0, un-connect: 6
 - Wisentic_Error: 15, connect:15, un-connect: 0
 - Other: 2, connect:0, un-connect: 2
 - Information: 0, connect:0, un-connect: 0
 - Insurance_Validation_Error: 5, connect:4, un-connect: 1
 - Settlement_Denied: 4, connect:4, un-connect: 0
 - Settlement_Approved: 506, connect:498, un-connect: 8
 - Settlement_Request: 7, connect:0, un-connect: 7
 - Message: 7, connect:6, un-connect: 1
 - Question: 8, connect:5, un-connect: 3
 - Spam: 1, connect:0, un-connect: 1
 - Complement_DR_Insurance_Company: 49, connect:49, un-connect: 0
 - Complement_Damage_Request_Insurance_Company: 56, connect:2, un-connect: 54
 - Complement_DR_Clinic: 57, connect:57, un-connect: 0
 - Complement_Da

{'total_records': 858,
 'auto_categorized': '810 (94.4%)',
 'manual_handling_required': '48 (5.6%)',
 'connected_to_errands': '710 (82.8%)',
 'single_errand_connection': '710 (82.8%)',
 'reliable_connections': '622 (72.5%)',
 'staff_animals': '0 (0.0%)'}

In [5]:
from app.schemas.forward import ForwardingIn
from app.services.services import DefaultServices
from app.dataset.forwarding_dataset import ForwardingEmailDataset
id = 182881
forwarding_request = ForwardingIn(id=id, userId=14)
ds = ForwardingEmailDataset(services=DefaultServices())
forward = ds.do_forwarding(forwarding_request)
model_to_dataframe([forward])

,id,forward_address,forward_subject,forward_text
0,182881,info@dinglevet.se,Info från försäkringsbolaget: Direktreglering ...,"Hej,<br><br>Här kommer önskemål om kompletteri..."


In [1]:
from app.services.utils import fetchFromDB, dataframe_to_model, tz_convert, model_to_dataframe
from app.dataset.payment_dataset import PaymentDataset
from app.services.payment_service import PaymentService
from app.services.base_service import BaseService
from app.schemas.payment import PaymentIn
import pandas as pd
bs = BaseService()
ds = PaymentDataset()
ps = PaymentService()

# id = 63943
# pay = fetchFromDB((bs.queries['payment'].iloc[0]).format(ID=id))
pay = pd.read_json("data/test_data/pay.json")

payIn = dataframe_to_model(pay, PaymentIn)
a = ds.match_payments(payIn)
df = model_to_dataframe(a)

ps.statistics(df)

🚀 Starting payment processing for 663 payments...
📊 Pre-loading database data...
✅ Database loaded in 2.35s
✅ Step 1 (Process): 0.01s
✅ Step 2 (Parse info): 0.15s
✅ Step 3 (Match by info): 8.66s
✅ Step 4 (Reminder unmatched): 342.98s
✅ Step 5 (Match entity/amount): 4.90s
✅ Step 6 (Match payouts): 0.69s
✅ Format amounts: 0.00s

⏱️ Total processing time: 359.74s (1.8 payments/sec)

🎯 PERFECT MATCHED PAYMENT IDs (8 total):
   [63580, 63604, 63953, 63972, 64058, 64076, 64083, 64126]


{'total': np.int64(663),
 'matched': np.int64(12),
 'matched_rate': np.float64(1.809954751131222),
 'perfect_matched': np.int64(8),
 'perfect_rate': np.float64(1.206636500754148),
 'paid_out': np.int64(0),
 'paid_out_rate': np.float64(0.0),
 'unmatched': np.int64(651),
 'unmatched_rate': np.float64(98.19004524886877)}

In [7]:
df.loc[df['status'].str.contains('relevant'),'id'].count()

np.int64(4)

In [ ]:
id = 63943
bs = BaseService()
ps = PaymentService()
pay = fetchFromDB((bs.queries['payment'].iloc[0]).format(ID=id))
pay = pd.read_json("data/test_data/pay.json")
print(pay)
payIn = dataframe_to_model(pay, PaymentIn)
ds = PaymentDataset()
a = ds.match_payments(payIn)

,id,amount,reference,info,bankName,createdAt,insuranceCaseId,status,valPay,valErrand,isReference,referenceLink,settlementAmount,extractReference,extractDamageNumber,extractOtherNumber
136,63943,10745.00 kr,,SKADENUMMER: 253919770\n\nFAKTURA\n\n100073862...,IF SKADEFÖRSÄKRING AB (PUBL),2025-09-01 14:28:59.885000+02:00,[],"Found 1 relevant DR (reference: <a href=""https...",[],[],[1000751840],"[<a href=""https://admin.direktregleringsportal...",2646900.0,None,253919770,1000738623


In [21]:
er = fetchFromDB((bs.queries['errandConnect'].iloc[0]).format(CONDITION="er.\"createdAt\" BETWEEN '2025-01-01' AND '2025-12-31' AND c.\"name\" ILIKE '%Hälsinge Smådjursklinik Hudiksvall%'"))
print(er.columns)
groups = er.groupby(['ownerName', 'animalName', 'insuranceCompany'])
result = []
i = 1
for key, group in groups:
    group_sorted = group.sort_values('date')
    group_sorted['diff_days'] = group_sorted['date'].diff().dt.days
    filtered = group_sorted[group_sorted['diff_days'] <= 5]
    if not filtered.empty and (i == 1):
        print("group_sorted:\n", group_sorted, "\nfiltered: \n", filtered)
        i += 1

    if not filtered.empty:
        result.append(filtered)

result_df = pd.concat(result) if result else pd.DataFrame()

print(result_df)


Index(['errandId', 'errandNumber', 'date', 'insuranceCompany', 'clinicName',
       'totalAmount', 'settlementAmount', 'reference', 'insuranceNumber',
       'damageNumber', 'invoiceReference', 'animalName', 'ownerName',
       'paymentOption', 'strategyType', 'settled'],
      dtype='object')
group_sorted:
      errandId errandNumber                             date insuranceCompany  \
608     68709  16220687095 2025-06-09 13:54:56.335324+00:00               If   
587     68983  16220689836 2025-06-12 08:07:48.508955+00:00               If   
115     74017  16220740175 2025-08-19 12:56:19.137794+00:00               If   

                             clinicName  totalAmount  settlementAmount  \
608  Hälsinge Smådjursklinik Hudiksvall       1592.0            1353.0   
587  Hälsinge Smådjursklinik Hudiksvall       4374.0            3718.0   
115  Hälsinge Smådjursklinik Hudiksvall       2226.0            1748.0   

      reference   insuranceNumber damageNumber invoiceReference animalNa

In [98]:
import pandas as pd
from datetime import timedelta
query = (f"er.\"createdAt\" BETWEEN '2025-01-01' AND '2025-12-31'" 
         f"AND c.\"name\" ILIKE '%Hälsinge Smådjursklinik Hudiksvall%'"
         f"AND es.cancelled = FALSE")
er = fetchFromDB((bs.queries['errandConnect'].iloc[0]).format(CONDITION=query))
WINDOW = pd.Timedelta(days=5)

def fmt_delta(td: pd.Timedelta) -> str:
    total_seconds = int(td.total_seconds())
    if total_seconds < 0:
        total_seconds = -total_seconds
    days, rem = divmod(total_seconds, 24*3600)
    hours, rem = divmod(rem, 3600)
    minutes, _ = divmod(rem, 60)
    parts = []
    if days:   parts.append(f"{days}day")
    if hours:  parts.append(f"{hours}hour")
    if minutes:parts.append(f"{minutes}min")
    if not parts:  # 小于1分钟
        parts = ["0min"]
    return "".join(parts)

er['createdAt'] = pd.to_datetime(er['date'], errors='coerce', utc=True).dt.tz_convert('Europe/Stockholm')
filtered_list = []

for key, grp in er.groupby(['ownerName', 'animalName', 'insuranceCompany'], dropna=False):
    g = grp.sort_values('createdAt').copy()
    g['delta_to_prev'] = g['createdAt'].diff()
    mask = g['delta_to_prev'] <= WINDOW
    if not mask.any():
        continue

    g['gap_label'] = ""
    g.loc[mask, 'gap_label'] = g.loc[mask, 'delta_to_prev'].apply(fmt_delta)
    prev_idx = g.index[mask] - 1  
    prev_index = g.loc[mask].index.map(lambda i: g.index[g.index.get_loc(i)-1])

    for i in prev_index:
        if g.at[i, 'gap_label'] == "":
            g.at[i, 'gap_label'] = "first"

    filtered_list.append(g[g['gap_label'] != ""])

filtered = pd.concat(filtered_list, ignore_index=False) if filtered_list else pd.DataFrame()
filtered = filtered.sort_values(['ownerName','animalName','insuranceCompany','createdAt'])
filtered.head()

,errandId,errandNumber,date,insuranceCompany,clinicName,totalAmount,settlementAmount,reference,insuranceNumber,damageNumber,invoiceReference,animalName,ownerName,paymentOption,strategyType,settled,createdAt,delta_to_prev,gap_label
612,68709,16220687095,2025-06-09 13:54:56.335324+00:00,If,Hälsinge Smådjursklinik Hudiksvall,1592.0,1353.0,1000684066,KD223469-8393-01,255282131,11250006374,Jenna,Agneta Nyström,fortus,creditcheck,True,2025-06-09 15:54:56.335324+02:00,NaT,first
592,68983,16220689836,2025-06-12 08:07:48.508955+00:00,If,Hälsinge Smådjursklinik Hudiksvall,4374.0,3718.0,1000686756,KD223469-8393-01,255282131,11250006534,Jenna,Agneta Nyström,fortus,creditcheck,True,2025-06-12 10:07:48.508955+02:00,2 days 18:12:52.173631,2day18hour12min
57,75065,39433750658,2025-09-01 12:46:07.492328+00:00,Agria,Hälsinge Smådjursklinik Hudiksvall,2620.0,0.0,1000747384,1236985-003-001,7575739-1,11250009989,Mini,Anders Eriksson,fortus,creditcheck,True,2025-09-01 14:46:07.492328+02:00,NaT,first
54,75098,39433750989,2025-09-02 07:19:39.908932+00:00,Agria,Hälsinge Smådjursklinik Hudiksvall,699.0,594.0,1000747707,1236985-003-001,7576557-1,11250010009,Mini,Anders Eriksson,swedbankPay,settlement,True,2025-09-02 09:19:39.908932+02:00,0 days 18:33:32.416604,18hour33min
213,73031,38568730311,2025-08-06 07:56:10.429513+00:00,Folksam,Hälsinge Smådjursklinik Hudiksvall,11727.0,5976.0,1000727139,CV-2083026-993,FF093566214S,11250008912,Hilja,Anders Svahn,manual,settlement,True,2025-08-06 09:56:10.429513+02:00,NaT,first


In [99]:
e1 = filtered.loc[filtered['gap_label']=='first', ['errandNumber', 'date', 'insuranceNumber', 'damageNumber', 'invoiceReference','animalName','ownerName','insuranceCompany']]
e2 = filtered.loc[filtered['gap_label']!='first', ['errandNumber', 'date', 'insuranceNumber', 'damageNumber', 'invoiceReference','animalName','ownerName','insuranceCompany']]
e = e1.merge(e2, on=['animalName','ownerName','insuranceCompany'], how = 'left', suffixes=['_before','_after'])
e.loc[:,['insuranceNumber_consistant','damageNumber_consistant','invoiceReference_consistant']] = False

for idx, row in e.iterrows():
    if row['insuranceNumber_before'] == row['insuranceNumber_after']:
        e.at[idx, 'insuranceNumber_consistant'] = True  
    if row['damageNumber_before'] == row['damageNumber_after']:
        e.at[idx, 'damageNumber_consistant'] = True
    if row['invoiceReference_before'] == row['invoiceReference_after']:
        e.at[idx, 'invoiceReference_consistant'] = True
e[['animalName','ownerName','insuranceCompany','errandNumber_before','date_before','errandNumber_after','date_after','insuranceNumber_consistant','damageNumber_consistant','invoiceReference_consistant']].head()

,animalName,ownerName,insuranceCompany,errandNumber_before,date_before,errandNumber_after,date_after,insuranceNumber_consistant,damageNumber_consistant,invoiceReference_consistant
0,Jenna,Agneta Nyström,If,16220687095,2025-06-09 13:54:56.335324+00:00,16220689836,2025-06-12 08:07:48.508955+00:00,True,True,False
1,Mini,Anders Eriksson,Agria,39433750658,2025-09-01 12:46:07.492328+00:00,39433750989,2025-09-02 07:19:39.908932+00:00,True,False,False
2,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568732408,2025-08-08 07:04:51.851619+00:00,True,True,False
3,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568738041,2025-08-15 12:45:44.422810+00:00,True,True,False
4,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568738595,2025-08-18 09:09:54.390669+00:00,True,True,False


In [101]:
e['återbesök'] = [[] for i in range(len(e))] 
f = e.groupby(['animalName','ownerName','insuranceCompany','errandNumber_before'])
# for idx, r in f:
#     if r.errandNumber_before.count()>2:
#         print("=========== ",idx," =============\n",r)

In [102]:
g = e[['animalName','ownerName','insuranceCompany','errandNumber_before']]
g = g.drop_duplicates()
g.errandNumber_before.count()

np.int64(106)

In [103]:
query = '''SELECT
    c.name AS "Klinik",
    prev.reference AS "Ärende1",
    prev."createdAt" AS "date1",
    e1.reference AS "Ärende2",
    e1."createdAt" AS "date2"
    FROM clinic c
    JOIN errand e1 ON e1."clinicId" = c.id
    JOIN insurance_case ic1 ON ic1."errandId" = e1.id
    JOIN insurance_company_email ice1 ON ic1."insuranceCompanyEmailId" = ice1.id
    INNER JOIN LATERAL (
    SELECT e2.reference, e2."createdAt"
    FROM insurance_case ic2
    JOIN insurance_company_email ice2 ON ic2."insuranceCompanyEmailId" = ice2.id
    JOIN errand e2 ON ic2."errandId" = e2.id
    JOIN errand_status es ON es.id = e2."statusId"
    WHERE ic2."insuranceNumber" = ic1."insuranceNumber"
        AND ice2."insuranceCompanyId" = ice1."insuranceCompanyId"
        AND e2.id <> e1.id
        AND e2."clinicId" = e1."clinicId"
        AND e2."createdAt" <= e1."createdAt"
        AND e2."createdAt" >= (e1."createdAt" - INTERVAL '5 days')
        AND es."cancelled" IS FALSE
    ORDER BY e2."createdAt" DESC   -- closest prior
    LIMIT 1
    ) AS prev ON TRUE
    WHERE c.id = 38
    AND EXTRACT(YEAR FROM e1."createdAt") = 2025;'''
emy = fetchFromDB(query)
emy

,Klinik,Ärende1,date1,Ärende2,date2
0,Hälsinge Smådjursklinik Hudiksvall,32970599398,2025-02-07 13:53:12.700907+00:00,32970599760,2025-02-08 08:52:45.665570+00:00
1,Hälsinge Smådjursklinik Hudiksvall,16203595356,2025-02-04 10:01:11.796588+00:00,16203597279,2025-02-05 15:06:26.356796+00:00
2,Hälsinge Smådjursklinik Hudiksvall,25301588445,2025-01-27 09:58:29.341031+00:00,25301592561,2025-01-30 16:17:24.820264+00:00
3,Hälsinge Smådjursklinik Hudiksvall,31836571005,2025-01-02 14:52:16.571501+00:00,31836572227,2025-01-05 11:35:28.560724+00:00
4,Hälsinge Smådjursklinik Hudiksvall,32804594235,2025-02-03 10:57:57.780642+00:00,32804596065,2025-02-04 14:21:34.175280+00:00
...,...,...,...,...,...
133,Hälsinge Smådjursklinik Hudiksvall,39433750658,2025-09-01 12:46:07.492328+00:00,39433750989,2025-09-02 07:19:39.908932+00:00
134,Hälsinge Smådjursklinik Hudiksvall,39489751964,2025-09-02 14:39:31.892744+00:00,39489752095,2025-09-03 07:35:50.378746+00:00
135,Hälsinge Smådjursklinik Hudiksvall,39401750896,2025-09-01 15:05:06.010931+00:00,39401752371,2025-09-03 09:42:59.176342+00:00
136,Hälsinge Smådjursklinik Hudiksvall,39159753282,2025-09-04 08:59:42.567744+00:00,39159753308,2025-09-04 09:19:32.768594+00:00


In [106]:
h = emy.merge(e[['errandNumber_before', 'errandNumber_after']], left_on=['Ärende1', 'Ärende2'], right_on=['errandNumber_before', 'errandNumber_after'], how='left')
h.loc[h['Ärende1'].notna() & h['errandNumber_before'].isna()].sort_values('Ärende1').head()

,Klinik,Ärende1,date1,Ärende2,date2,errandNumber_before,errandNumber_after
20,Hälsinge Smådjursklinik Hudiksvall,13249583660,2025-01-20 15:42:24.710690+00:00,13249583991,2025-01-21 09:58:18.168951+00:00,NaN,NaN
137,Hälsinge Smådjursklinik Hudiksvall,13585750204,2025-09-01 09:36:32.337658+00:00,13585754404,2025-09-05 07:49:11.746104+00:00,NaN,NaN
30,Hälsinge Smådjursklinik Hudiksvall,15133615474,2025-02-28 13:15:58.916833+00:00,15133616381,2025-03-03 11:06:54.664510+00:00,NaN,NaN
34,Hälsinge Smådjursklinik Hudiksvall,15133616381,2025-03-03 11:06:54.664510+00:00,15133619104,2025-03-05 14:53:56.031371+00:00,NaN,NaN
33,Hälsinge Smådjursklinik Hudiksvall,15133619104,2025-03-05 14:53:56.031371+00:00,15133619344,2025-03-06 08:49:15.429581+00:00,NaN,NaN


In [107]:
import pandas as pd

def merge_chain(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    df = df.copy()
    result = []
    start_nodes = set(df[col1]) - set(df[col2])
    for start in start_nodes:
        chain = []
        current = start
        while True:
            row = df.loc[df[col1] == current]
            if row.empty:
                break
            next_val = row[col2].values[0]
            chain.append(next_val)
            current = next_val
        result.append({col1: start, col2: chain})
    return pd.DataFrame(result)

sub = merge_chain(emy, 'Ärende1','Ärende2')
sub

,Ärende1,Ärende2
0,25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025]
4,20380645281,[20380645935]
...,...,...
106,34273630219,[34273630235]
107,39205744988,[39205748344]
108,32970599398,[32970599760]
109,33736617037,[33736617300]


In [108]:
import pandas as pd
from typing import List, Optional

def compress_errand_sequences(
    df: pd.DataFrame,
    number_col: str = "errandNumber",
    label_col: str = "gap_label",
    id_col: Optional[str] = None,
    keep_extra_cols: Optional[List[str]] = None,
) -> pd.DataFrame:
    """
    将按序排列的链式多行压成一行：
      - 每个序列以 gap_label == 'first' 开始
      - 该序列后续行的 errandNumber 收集到 subsequence 列
      - 如果有多条链（多个 'first'），各自输出一行

    参数
    ----
    df : 输入DataFrame（应按时间/链顺序排好）
    number_col : 存放案件号的列名
    label_col : 存放标签（如 'first', '3day...'）的列名
    id_col : 可选，若存在则保留第一行该列
    keep_extra_cols : 可选，额外需要从第一行保留的列名列表

    返回
    ----
    DataFrame：列包含
      - 如果给了 id_col：该列
      - number_col：第一行的案件号
      - label_col：'first'
      - subsequence：list[str]，后续行的案件号（字符串形式）
    """
    if keep_extra_cols is None:
        keep_extra_cols = []

    required = [number_col, label_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    records = []
    current_group = None

    def flush_group():
        nonlocal current_group
        if current_group is not None:
            records.append(current_group)
            current_group = None

    # 逐行扫描，遇到 'first' 开启新组，其余加入 subsequence
    for _, row in df.iterrows():
        lbl = row[label_col]
        num = row[number_col]

        if lbl == "first":
            # 先把前一个组收尾
            flush_group()

            # 新组：以当前行为主行
            current_group = {
                number_col: num,
                label_col: "first",
                "subsequence": [],  # 后续行会填进来
            }
            if id_col and id_col in df.columns:
                current_group[id_col] = row[id_col]
            for c in keep_extra_cols:
                if c in df.columns:
                    current_group[c] = row[c]
        else:
            # 非 first：挂到当前组的 subsequence
            if current_group is None:
                # 若数据不规范（第一行就不是 first），则开启匿名组
                current_group = {
                    number_col: num,
                    label_col: lbl,
                    "subsequence": [],
                }
                if id_col and id_col in df.columns:
                    current_group[id_col] = row.get(id_col)
                for c in keep_extra_cols:
                    if c in df.columns:
                        current_group[c] = row[c]
            else:
                current_group["subsequence"].append(str(num))

    # 收尾
    flush_group()

    # 规范输出列顺序
    cols = []
    if id_col and (len(records) == 0 or id_col in records[0]):
        cols.append(id_col)
    cols += [number_col, label_col, "subsequence"]
    # 附加保留列（若有）
    for c in keep_extra_cols:
        if len(records) and c in records[0] and c not in cols:
            cols.append(c)

    out = pd.DataFrame(records)
    # 只保留存在的列，避免 KeyError
    cols = [c for c in cols if c in out.columns]
    return out[cols]
j = compress_errand_sequences(filtered)
j.head()

,errandNumber,gap_label,subsequence
0,16220687095,first,[16220689836]
1,39433750658,first,[39433750989]
2,38568730311,first,[38568732408]
3,38568735666,first,"[38568738041, 38568738595]"
4,33736617037,first,[33736617300]


In [109]:
g = sub.merge(j, left_on='Ärende1', right_on='errandNumber', how='left')
g

,Ärende1,Ärende2,errandNumber,gap_label,subsequence
0,25895600143,[25895603949],25895600143,first,[25895603949]
1,25301585904,"[25301588445, 25301592561]",25301585904,first,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]",35918673704,first,"[35918675147, 35918677861]"
3,33076600684,[33076604025],33076600684,first,[33076604025]
4,20380645281,[20380645935],20380645281,first,[20380645935]
...,...,...,...,...,...
106,34273630219,[34273630235],34273630219,first,[34273630235]
107,39205744988,[39205748344],39205744988,first,[39205748344]
108,32970599398,[32970599760],32970599398,first,[32970599760]
109,33736617037,[33736617300],33736617037,first,[33736617300]


In [110]:
for idx, row in g.iterrows():
    if row['Ärende2'] == row['subsequence']:
        g.at[idx,'eq'] = 'EQ'
g.loc[g['errandNumber'].isna()]

,Ärende1,Ärende2,errandNumber,gap_label,subsequence,eq
5,31727569761,[31727571528],NaN,NaN,NaN,NaN
31,39240745982,"[24064747736, 24064748940]",NaN,NaN,NaN,NaN
41,31770569445,[31770571995],NaN,NaN,NaN,NaN
60,37482706019,[37482706027],NaN,NaN,NaN,NaN
95,35187652082,[35187655044],NaN,NaN,NaN,NaN
100,34364632322,[34364634138],NaN,NaN,NaN,NaN


In [90]:
er.loc[er['errandNumber']=='31727569761']

,errandId,errandNumber,date,insuranceCompany,clinicName,totalAmount,settlementAmount,reference,insuranceNumber,damageNumber,invoiceReference,animalName,ownerName,paymentOption,strategyType,settled,createdAt


In [116]:
query = '''WITH
  errands_2025 AS (
    SELECT
      er.id,
      er.reference,
      er."createdAt",
      er."animalOwnerId",
      ic."insuranceNumber",
      ic."animalId"
    FROM errand er
    JOIN insurance_case ic ON ic."errandId" = er.id
    JOIN errand_status es ON es.id = er."statusId"
    WHERE
      er."clinicId" = 38
      AND er."createdAt" >= '2025-01-01'
      AND er."createdAt" < '2026-01-01'
      AND ic."insuranceNumber" IS NOT NULL
      AND ic."insuranceNumber" <> ''
      AND es."cancelled" IS FALSE
  ),
  paired_errands AS (
    SELECT
      e1."animalOwnerId",
      e1."insuranceNumber",
      e2.reference AS "errandNumber1",
      e2."createdAt" AS "date1",
      e1.reference AS "errandNumber2",
      e1."createdAt" AS "date2",
      e1."animalId"
    FROM
      errands_2025 e1
      JOIN errands_2025 e2 ON e1."animalOwnerId" = e2."animalOwnerId"
      AND e1."insuranceNumber" = e2."insuranceNumber"
      AND e1.id <> e2.id
      AND e1."createdAt" > e2."createdAt"
      AND e1."createdAt" <= e2."createdAt" + INTERVAL '5 days'
  )
    SELECT DISTINCT p."errandNumber1", p."date1", p."errandNumber2", p."date2",
    a.name AS "animalName",
    CONCAT(ao."firstName", ' ', ao."lastName") AS "ownerName",
    p."insuranceNumber"
    FROM paired_errands p
    JOIN animal a ON a.id = p."animalId"
    JOIN animal_owner ao ON ao.id = p."animalOwnerId"
    ORDER BY p."errandNumber1";'''
line = fetchFromDB(query)
l = merge_chain(line, 'errandNumber1', 'errandNumber2')
l

,errandNumber1,errandNumber2
0,25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025]
4,20380645281,[20380645935]
...,...,...
103,34273630219,[34273630235]
104,39205744988,[39205748344]
105,32970599398,[32970599760]
106,33736617037,[33736617300]


In [128]:
l.loc[l['errandNumber2'].apply(len)>1]

,errandNumber1,errandNumber2
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
6,38153725148,"[38153727391, 38153728829, 38153729835]"
21,35842666782,"[35842668085, 35842668945]"
24,22827574983,"[22827576814, 22827580493]"
29,15133614394,"[15133615474, 15133616381, 15133619104, 151336..."
40,35721664072,"[35721665301, 35721666770]"
44,38841736317,"[38841737315, 38841737547]"
45,36237676063,"[36237677095, 36237677202]"
53,35208658704,"[35208660106, 35208662847]"


In [132]:
l.loc[l['errandNumber1']=='15133614394','errandNumber2'].to_list()

[['15133615474', '15133616381', '15133619104', '15133619344', '15133622488']]

In [125]:
g = l.merge(sub, left_on='errandNumber1', right_on='Ärende1', how='left')
g

,errandNumber1,errandNumber2,Ärende1,Ärende2
0,25895600143,[25895603949],25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]",25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]",35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025],33076600684,[33076604025]
4,20380645281,[20380645935],20380645281,[20380645935]
...,...,...,...,...
103,34273630219,[34273630235],34273630219,[34273630235]
104,39205744988,[39205748344],39205744988,[39205748344]
105,32970599398,[32970599760],32970599398,[32970599760]
106,33736617037,[33736617300],33736617037,[33736617300]
